In [1]:
from pyTigerGraph import TigerGraphConnection
import torch

In [2]:
def load_data(dataloader):
    data = []
    for epoch in range(2):
        for batch in dataloader:
            data.append(batch)
    return data

In [3]:
# conn1 is from machine 1
conn1 = TigerGraphConnection(
    host="http://35.230.92.92", 
    graphname="Cora")

In [4]:
conn1.gds.configureKafka(
    kafka_address="34.82.171.137:9092",
    kafka_auto_del_topic=False,
    kafka_skip_produce=False,
    kafka_add_topic_per_epoch=True,
    kafka_del_topic_per_epoch=False,
)

In [4]:
conn1.gds.configureKafka(
    kafka_address="34.82.171.137:9092",
)

In [5]:
dataloader1 = conn1.gds.neighborLoader(
    v_in_feats=["x"],
    v_out_labels=["y"],
    batch_size=16,
    num_neighbors=10,
    num_hops=2,
    shuffle=True,
    filter_by="train_mask",
    output_format="PyG",
    loader_id="test_multi_consumer",
)

In [6]:
res1 = load_data(dataloader1)

In [7]:
# conn2 is from machine 2
conn2 = TigerGraphConnection(
    host="http://35.230.92.92", 
    graphname="Cora")

In [8]:
conn2.gds.configureKafka(
    kafka_address="34.82.171.137:9092",
    kafka_auto_del_topic=False,
    kafka_skip_produce=True,
    kafka_add_topic_per_epoch=True,
    kafka_del_topic_per_epoch=False
)

In [9]:
dataloader2 = conn2.gds.neighborLoader(
    v_in_feats=["x"],
    v_out_labels=["y"],
    batch_size=16,
    num_neighbors=10,
    num_hops=2,
    shuffle=True,
    filter_by="train_mask",
    output_format="PyG",
    loader_id="test_multi_consumer",
)

In [10]:
res2 = load_data(dataloader2)

In [20]:
# Check if the two connections get the same data
assert len(res1) == len(res2)

for i,j in zip(res1, res2):
    assert torch.allclose(i.edge_index, j.edge_index)
    assert torch.allclose(i.x, j.x)
    assert torch.allclose(i.y, j.y)
    assert torch.allclose(i.is_seed, j.is_seed)